In [3]:
import pandas as pd
import math as m

In [4]:
# Mise en forme des données

import pandas as pd
import numpy as np


def file_to_dataframe(filenames,st): 
    """
    Initialize the DataFrame from a filenames 
    Parameters: 
    -----------
    filenames: str, the name of the file
    st: str, helps to delimites the columns of the datas because "Correspondances_promo
    are delimited with ';' while the other are delimited with ','.
    """
    datas = pd.read_csv(filenames,delimiter=st)
    return datas


def clean_dates(df):
    """
    Clean the dates of the files with dates such as "2022-11-14T00:00:00Z" and change it into "2022-11-14"
    -----------
    df: dataframe, the dataframe that we use
    
    """
    for colu in df.columns :
        if df[colu].dtype == object: 
            df[colu] = df[colu].str.replace('T00:00:00Z', '')
    return(df)

def change_date(st):
    """
    Change a date as "28/04/2003 00:00:00.000" into one more precise "2003-04-28"
    -----------
    st: str, the date to change

    """ 
    if type(st) != str or len(st) < 10 : 
        return(st)
    else :
        date = st[:10].split("/")
        return(date[2]+"-"+date[1]+"-"+date[0])
    
def change_dates_all(df,cols):
    """
    Change the dates of some columns of a dataFrame using the function change_date
    -----------
    df: dataframe, the DataFrame on which we are working
    cols: str list, a list of columns name that we want to change

    """ 
    for colu in cols : 
        df[colu] = df[colu].apply(change_date)
    return df

def save_to_csv_file(df,filename):
    """
    Save a dataframe on a .csv file at filename
    ----------
    df: dataframe, the DataFrame that you want to save
    filename: str, the place where you want to save your data frame
    """
    df.to_csv(filename, index=False)
    return True

In [5]:

#1)

def df_filter_condition(df,str_name, str_cond):
    """
    This function filters the elemenent of the columns str_name with the condition ==
    str_cond
    """
    return df[df[str_name] == str_cond]


def df_filter_begin_name(df, str_name, str_filter):
    """
    This function filters the elemenent of the columns str_name which begins with 
    str_filter
    """
    return df[df[str_name].str.startswith(str_filter)]


def apply_conditions(row):
    """
    This function creates the elements of a new column where the promos are put
    together in function of some conditions.
    """
    nb_days = 'NBJOUR_ODD'
    carticle = 'CARTICLE_ODD'
    if row['SG'] == 'Semaine généreuse':
        return 'Semaine genéreuse'
    elif row[nb_days] == 7 :
        return 'ODD 7 jours autre que SG'
    elif (row[carticle][:2] == 'EV') & (row[nb_days] == 15) :
        return 'ODD 15 jours EV+'
    elif (row[carticle][:2]  == 'TC') & (row[nb_days] == 15) :
        return 'ODD 15 jours TC'
    elif (row[carticle][:2] == 'EV') & (row[nb_days] == 21) :
        return 'ODD 21 jours EV+ '
    elif (row[carticle][:2] == 'TC') & (row[nb_days] == 21):
        return 'ODD 21 jours EV+ '
    else: 
        return 'Autres'
        
def create_new_column(df: pd.DataFrame,function):
    """
    This function is used to sendback a new column using the function apply
    and applying function to the elements 
    """
    return df.apply(function,axis = 1)

def keep_used_odd(df_Données_Promos,df_odd,n) :
    """
    This function delete all the promos which are not used enough
    """
    df_join_odd = join_dataFrames(df_Données_Promos,df_odd,['CPROMO'])

    series_count = df_join_odd.groupby(['CPROMO'])['ID_ABONNE'].count() #count the number of people who used each PROMO
    series_count = series_count.sort_values(ascending=False) #sort the number of people
    series_count = series_count[series_count > n ] # Keep the more used
    series_count = series_count.rename('NOMBRE_UTILISATION') # Rename with a proper name

    df_filtered_groups = series_count.reset_index() # series to Data fram
    df_new_odd = join_dataFrames(df_filtered_groups,df_odd,['CPROMO'])

    return df_new_odd

def str_to_date(df,col_name):
    return pd.to_datetime(df[col_name])



In [10]:
#Test

#1)

def creation_df_odd() :
    """
    This function creates the new df_odd on your Computer
    """
    df_Correspondances_Promos = file_to_dataframe("/Users/antoine/Documents/ENSAE 2A/Code perso/everything/Statapp perso/Ressources/Correspondances_Promos.csv",",") 

    df_odd = df_filter_condition(df_Correspondances_Promos,'TYPE_PROMO','ODD') #we create a DataFrame with only ODD Promotion
    df_odd['TYPE_PROMON'] = create_new_column(df_odd,apply_conditions) #we create new columns on this DataFrame of the ODD type


    #creation df_odd_2021
    df_Données_Promos_2021 = file_to_dataframe("/Users/antoine/Documents/ENSAE 2A/Code perso/everything/Statapp perso/Ressources/Données_Promos_2021.csv",",")

    n = df_Données_Promos_2021.shape[0] / 10000 #number minimum of used
    df_new_odd = keep_used_odd(df_Données_Promos_2021,df_odd,n) #creation of the new tab by keeping only the used promos
    
    save_to_csv_file(df_new_odd,"/Users/antoine/Documents/ENSAE 2A/Code perso/everything/Statapp perso/Ressources/odd_2021.csv")

    #creation df_odd_2022  
    df_Données_Promos_2022 = file_to_dataframe("/Users/antoine/Documents/ENSAE 2A/Code perso/everything/Statapp perso/Ressources/Données_Promos_2022.csv",",")

    n = df_Données_Promos_2022.shape[0] / 10000 #number minimum of used
    df_new_odd = keep_used_odd(df_Données_Promos_2022,df_odd,n) #creation of the new tab by keeping only the used promos
    
    save_to_csv_file(df_new_odd,"//Users/antoine/Documents/ENSAE 2A/Code perso/everything/Statapp perso/Ressources/odd_2022.csv")
    
    #creation df_odd_2023
    df_Données_Promos_2023 = file_to_dataframe("/Users/antoine/Documents/ENSAE 2A/Code perso/everything/Statapp perso/Ressources/Données_Promos_2023.csv",",")

    n = df_Données_Promos_2023.shape[0] / 10000 #number minimum of used
    df_new_odd = keep_used_odd(df_Données_Promos_2023,df_odd,n) #creation of the new tab by keeping only the used promos
    
    save_to_csv_file(df_new_odd,"/Users/antoine/Documents/ENSAE 2A/Code perso/everything/Statapp perso/Ressources/odd_2023.csv")

    return True

In [1]:
# 2

# Filtrer par type de promo, nouvelle colonne ? 

def df_mois_annee(df) :
    str_to_date(df,'DATE_ACTE_REEL')
    df['MONTH'] = df['DATE_ACTE_REEL'].dt.month

    return df

# Exo 2 Test pour 'CANAL_DISTRIB' à faire en +  : Enregistrer sur un csv + run pour les autres conditions 

def repartition_reabo(str):

    df_Données_Promos_2021 = file_to_dataframe("/Users/antoine/Documents/ENSAE 2A/Code perso/everything/Statapp perso/Ressources/Données_Promos_2021.csv")
    df_Données_Reabos_2021 = file_to_dataframe("/Users/antoine/Documents/ENSAE 2A/Code perso/everything/Statapp perso/Ressources/Données_Reabos_2021.csv")
    df_odd = file_to_dataframe("/Users/antoine/Documents/ENSAE 2A/Code perso/everything/Statapp perso/Ressources/odd_2021.csv")
    df_odd_new = df_odd[['CPROMO','TYPE_PROMON']] 

    df_join = join_dataFrames(df_Données_Promos_2021,df_odd_new,['CPROMO'])
    df_join = join_dataFrames(df_join,df_Données_Reabos_2021,['ID_ABONNE','DATE_ACTE_REEL'])
    df_join = df_mois_annee(df_join)

    series = df_join.groupby(['TYPE_PROMON','MONTH', str])['ID_ABONNE'].count()
    df_filtered_groups = series.reset_index(name = 'NB')

    save_to_csv_file(df_filtered_groups,"/Users/antoine/Documents/ENSAE 2A/Code perso/everything/Statapp perso/Ressources/repartition_reabo.csv")

    return True

In [ ]:

"""
print(df_join.groupby(['TYPE_PROMON','CANAL_DISTRIB'])['ID_ABONNE'].count())
print(df_join.groupby(['MONTH','CANAL_DISTRIB'])['ID_ABONNE'].count())
print(df_join.groupby(['TYPE_PROMON','MONTH','CANAL_DISTRIB'])['ID_ABONNE'].count())
"""   


In [ ]:
#3)

def str_to_date(df,col_name):
    return pd.to_datetime(df[col_name])


def time_reabo_columns(df,end_abo,date_reabo):
    """
    This function creates a column which represents the delay between the end 
    of the last abo and  the act of reabo
    """
    df[date_reabo] = str_to_date(df,date_reabo)
    df[end_abo] = str_to_date(df,end_abo)
    return (df[date_reabo] - df[end_abo]).dt.days

def count_abo_conditions(df,name_col,cond):
    """
    This function count the number of elements of the condition and group by
    the column 
    """
    return df.groupby(name_col)[cond].count()

def mean_time_reabo(df,name_col,cond):
    """
    This function calculate the mean of the condition and group by
    the column 
    """
    return df.groupby([name_col])[cond].mean()

def mean_empty_col(df,col,cond):
    """
    This function calculate the mean of the datas of column col by dividing
    between the datas where the columns cond is empty or not    
    """
    groupes = df.groupby(df[col].isna())
    return groupes[cond].mean()



In [ ]:

# Application 

def repartition_reabo():

    df_Données_Promos_2021 = file_to_dataframe("/Users/maximecoppa/Desktop/Statapp/Datas_clean/df_Données_Promos_2021.csv",",")
    df_Données_Reabos_2021 = file_to_dataframe("/Users/maximecoppa/Desktop/Statapp/Datas_clean/df_Données_Reabos_2021.csv",",")

    end_abo = 'DATE_FIN_ABO_PREC'
    date_reabo = 'DATE_ACTE_REEL'

    df_Données_Reabos_2021['TIME_REABO'] = time_reabo_columns(df_Données_Reabos_2021,end_abo,date_reabo) #Creation new colum "TIME_REABO"

    df_Données_Promos_2021['DATE_ACTE_REEL'] = str_to_date(df_Données_Promos_2021,'DATE_ACTE_REEL') #Preparation join df_Données_Promos_2021 et df_Données_Reabos_2021
    df_join = join_dataFrames(df_Données_Promos_2021,df_Données_Reabos_2021,['ID_ABONNE','DATE_ACTE_REEL']) #Join df_Données_Promos_2021 et df_Données_Reabos_2021

    series_count = mean_time_reabo(df_join,'CPROMO','TIME_REABO') #Creation of a Serie with the mean of the Reabo by CPROMO
    df_filtered_groups = series_count.reset_index() #Changing Series into a  DataFrame 
    save_to_csv_file(df_filtered_groups,"/Users/maximecoppa/Desktop/Statapp/Datas/mean_time_reabo_promos.csv") # Save on My Mac

    return True

def comparaison_time_reabo():


    df_Données_Promos_2021 = file_to_dataframe("/Users/maximecoppa/Desktop/Statapp/Datas_clean/df_Données_Promos_2021.csv",",")
    df_Données_Reabos_2021 = file_to_dataframe("/Users/maximecoppa/Desktop/Statapp/Datas_clean/df_Données_Reabos_2021.csv",",")

    end_abo = 'DATE_FIN_ABO_PREC'
    date_reabo = 'DATE_ACTE_REEL'

    df_Données_Reabos_2021['TIME_REABO'] = time_reabo_columns(df_Données_Reabos_2021,end_abo,date_reabo) #Creation new colum "TIME_REABO"

    df_Données_Promos_2021['DATE_ACTE_REEL'] = str_to_date(df_Données_Promos_2021,'DATE_ACTE_REEL') #Preparation join df_Données_Promos_2021 et df_Données_Reabos_2021

    df_join = join_dataFrames_outer(df_Données_Promos_2021,df_Données_Reabos_2021,['ID_ABONNE','DATE_ACTE_REEL']) #Join df_Données_Promos_2021 et df_Données_Reabos_2021 keeping the empty cases

    print(mean_empty_col(df_join,'CPROMO','TIME_REABO')) #Mean time for reabo with and without promotions False mean is promo True mean Na, On peut présenter mieux

    return True


In [22]:
# 4
#Répartition des réabonnements par taux de consommation (12 mois glissants) par type de promotions et hors promotion

def taux_consommation(file_path):
    # Charger le DataFrame à partir du fichier CSV
    df_Données_Reabos_2021 = pd.read_csv(file_path)

    # Compter le nombre de fois où chaque abonné est abonné
    series_count = df_Données_Reabos_2021.groupby(['ID_ABONNE'])['ID_ABONNE'].count()

    # Créer un DataFrame avec le nombre d'abonnements par abonné
    df_filtered_groups = series_count.reset_index(name='NB_ABOS')

    # Compter le nombre d'abonnés pour chaque nombre d'abonnements
    count_nb_abos = df_filtered_groups['NB_ABOS'].value_counts()
    count_nb_abos.columns = ['NB_ABOS', 'COUNT_NB_ABOS']

    # Calculer le taux de consommation ou effectuer d'autres opérations selon vos besoins
    # ...

    # Retourner le résultat ou ajuster selon ce que vous souhaitez obtenir
    return count_nb_abos.sort_values(ascending=False)


In [23]:
print(taux_consommation("/Users/antoine/Documents/ENSAE 2A/Code perso/everything/Statapp perso/Ressources/Données_Reabos_2021.csv"))

NB_ABOS
12    134279
1      69157
11     62490
2      48940
3      44605
4      40741
5      39065
10     37931
6      36469
9      33441
8      33402
7      32422
Name: count, dtype: int64
